# SGD regressor on the complete set of features

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, ParameterGrid
from sklearn.metrics import r2_score, accuracy_score, mean_absolute_error, classification_report, mean_squared_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

# Dataset

In [2]:
dataset = pd.read_csv("IDF.csv")

In [3]:
print("Number of rows : {}".format(dataset.shape[0]))
print("Number of columns : {}".format(dataset.shape[1]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 3643
Number of columns : 42

Display of dataset: 


,Unnamed: 0,Date,Code INSEE région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Bioénergies (MW),...,lag_8_IDF,lag_9_IDF,lag_10_IDF,lag_11_IDF,lag_12_IDF,lag_13_IDF,lag_14_IDF,lag_15_IDF,rolling_mean_7_IDF,rolling_mean_15_IDF
0,180,2013-01-16,11,610978.0,49163.0,0.0,75.0,124.0,0.0,5671.0,...,525381.0,496093.0,430898.0,433732.0,470053.0,487111.0,492157.0,399392.0,546067.285714,513250.266667
1,192,2013-01-17,11,640670.0,62705.0,0.0,7.0,157.0,0.0,5174.0,...,540858.0,525381.0,496093.0,430898.0,433732.0,470053.0,487111.0,492157.0,562741.571429,523151.133333
2,204,2013-01-18,11,653509.0,78732.0,0.0,197.0,49.0,27.0,5859.0,...,523950.0,540858.0,525381.0,496093.0,430898.0,433732.0,470053.0,487111.0,582288.714286,534244.333333
3,216,2013-01-19,11,593948.0,52654.0,0.0,212.0,0.0,10.0,6116.0,...,516679.0,523950.0,540858.0,525381.0,496093.0,430898.0,433732.0,470053.0,596959.142857,542504.000000
4,228,2013-01-20,11,580127.0,52397.0,0.0,202.0,0.0,47.0,5965.0,...,491255.0,516679.0,523950.0,540858.0,525381.0,496093.0,430898.0,433732.0,610503.714286,552263.666667



Basics statistics: 


,Unnamed: 0,Date,Code INSEE région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Bioénergies (MW),...,lag_8_IDF,lag_9_IDF,lag_10_IDF,lag_11_IDF,lag_12_IDF,lag_13_IDF,lag_14_IDF,lag_15_IDF,rolling_mean_7_IDF,rolling_mean_15_IDF
count,3643.000000,3643,3643.0,3643.000000,3643.000000,3643.0,3643.000000,3643.000000,3643.000000,3643.000000,...,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000,3643.000000
unique,NaN,3643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2013-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,22030.200659,NaN,11.0,383748.745265,12240.302086,0.0,762.985452,566.681993,282.004941,6646.680071,...,384143.445924,384174.210953,384184.308811,384191.703129,384243.331046,384287.822262,384327.178836,384334.817733,383925.966433,384076.104703
std,12618.407396,NaN,0.0,89603.608030,14724.274416,0.0,903.514750,457.992555,152.024934,996.070668,...,89826.685555,89845.832658,89849.017004,89851.986660,89847.270877,89858.070587,89873.911769,89874.009265,85270.226746,83482.566042
min,180.000000,NaN,11.0,23480.000000,-816.000000,0.0,0.000000,0.000000,0.000000,3413.000000,...,23480.000000,23480.000000,23480.000000,23480.000000,23480.000000,23480.000000,23480.000000,23480.000000,249972.142857,252030.733333
25%,11106.000000,NaN,11.0,312874.000000,0.000000,0.0,168.000000,218.000000,192.000000,6002.500000,...,312884.000000,312884.000000,312884.000000,312884.000000,312902.000000,312902.000000,312902.000000,312902.000000,315521.785714,315512.800000
50%,22032.000000,NaN,11.0,360363.000000,7195.000000,0.0,422.000000,481.000000,288.000000,6651.000000,...,360492.000000,360492.000000,360492.000000,360492.000000,360729.000000,360784.000000,360920.000000,360920.000000,358526.142857,359863.533333
75%,32958.000000,NaN,11.0,451039.500000,20132.500000,0.0,978.500000,764.000000,396.000000,7259.000000,...,451701.500000,451781.500000,451781.500000,451781.500000,451846.000000,451921.000000,452024.500000,452024.500000,453479.642857,457967.033333



Percentage of missing values: 


Unnamed: 0              0.000000
Date                    0.000000
Code INSEE région       0.000000
Consommation (MW)       0.000000
Thermique (MW)          0.000000
Nucléaire (MW)          0.000000
Eolien (MW)             0.000000
Solaire (MW)            0.000000
Hydraulique (MW)        0.000000
Bioénergies (MW)        0.000000
Ech. physiques (MW)     0.000000
Stockage batterie       0.000000
brent_price            30.496843
year                    0.000000
month                   0.000000
prix_kwh_elec          10.183914
temp_max                0.082350
temp_min                0.082350
hours_of_sun            0.109800
precipitation           0.109800
windspeed               0.082350
prix_gaz                0.164699
day                     0.000000
day_of_week             0.000000
gas_key                 0.000000
lag_1_IDF               0.000000
lag_2_IDF               0.000000
lag_3_IDF               0.000000
lag_4_IDF               0.000000
lag_5_IDF               0.000000
lag_6_IDF 

In [4]:
dataset.columns

Index(['Unnamed: 0', 'Date', 'Code INSEE région', 'Consommation (MW)',
       'Thermique (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)',
       'Hydraulique (MW)', 'Bioénergies (MW)', 'Ech. physiques (MW)',
       'Stockage batterie', 'brent_price', 'year', 'month', 'prix_kwh_elec',
       'temp_max', 'temp_min', 'hours_of_sun', 'precipitation', 'windspeed',
       'prix_gaz', 'day', 'day_of_week', 'gas_key', 'lag_1_IDF', 'lag_2_IDF',
       'lag_3_IDF', 'lag_4_IDF', 'lag_5_IDF', 'lag_6_IDF', 'lag_7_IDF',
       'lag_8_IDF', 'lag_9_IDF', 'lag_10_IDF', 'lag_11_IDF', 'lag_12_IDF',
       'lag_13_IDF', 'lag_14_IDF', 'lag_15_IDF', 'rolling_mean_7_IDF',
       'rolling_mean_15_IDF'],
      dtype='object')

# Train test split

In [5]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")

split_date = '2021-01-01'
train = dataset.loc[dataset['Date'] <= split_date].copy()
test = dataset.loc[dataset['Date'] > split_date].copy()

Dividing into train and test sets...


In [6]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = [
        "day_of_week", "temp_max","temp_min","hours_of_sun", "precipitation", "windspeed","prix_kwh_elec",'prix_gaz', 'brent_price', "lag_1_IDF", "lag_2_IDF",
        "lag_3_IDF", "lag_4_IDF", "lag_5_IDF", "lag_6_IDF", "lag_7_IDF",
        "lag_8_IDF", "lag_9_IDF", "lag_10_IDF", "lag_11_IDF", "lag_12_IDF",
        "lag_13_IDF", "lag_14_IDF", "lag_15_IDF", "rolling_mean_7_IDF",
        "rolling_mean_15_IDF"
                ]
target_variable = ["Consommation (MW)"]

X_train = train.loc[:,features_list]
X_test = test.loc[:,features_list]

y_train = train.loc[:,target_variable]
y_test = test.loc[:,target_variable]

Separating labels from features...


# Preprocessing

In [7]:
numeric_features = ["temp_max","temp_min", "hours_of_sun", "precipitation", "windspeed","prix_kwh_elec",'prix_gaz', 'brent_price',"lag_1_IDF", "lag_2_IDF",
       "lag_3_IDF", "lag_4_IDF", "lag_5_IDF", "lag_6_IDF", "lag_7_IDF",
       "lag_8_IDF", "lag_9_IDF", "lag_10_IDF", "lag_11_IDF", "lag_12_IDF",
       "lag_13_IDF", "lag_14_IDF", "lag_15_IDF", "rolling_mean_7_IDF",
       "rolling_mean_15_IDF"]
categorical_features = ["day_of_week"]

In [8]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=1)),
    ('scaler', StandardScaler()) 
])

# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('imputer', KNNImputer(n_neighbors=1)),
    ('encoder', OneHotEncoder(drop='first')),
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [9]:
# Preprocessings 
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test) 

# Model

In [10]:
sgdr = SGDRegressor(random_state=0)

In [11]:
param_grid = {
            'penalty': ['l1'],
            #'alpha' : [0.00025],
            'max_iter' : [5000]
            }

best_train_score = 0
best_test_score = 0

for g in ParameterGrid(param_grid):
    sgdr.set_params(**g)
    sgdr.fit(X_train, y_train.values.ravel())
    train_score = sgdr.score(X_train, y_train.values.ravel())
    test_score = sgdr.score(X_test, y_test.values.ravel())
    # if we get a better score, store the score and parameters
    if test_score > best_test_score:
        best_train_score = train_score
        best_test_score = test_score
        best_parameters = g

print("Best train score: {}".format(best_train_score))
print("Best test score: {}".format(best_test_score))
print("Best parameters: {}".format(best_parameters))

Best train score: 0.9927281129587657
Best test score: 0.9770554867391931
Best parameters: {'max_iter': 5000, 'penalty': 'l1'}


In [12]:
# Predictions
y_train_pred = sgdr.predict(X_train)
y_test_pred = sgdr.predict(X_test)

In [13]:
# MAPE
print("MSE on training set : ", mean_squared_error(y_train, y_train_pred))
print("MSE on test set : ", mean_squared_error(y_test, y_test_pred))
print()

print("MAPE on training set : ", mean_absolute_percentage_error(y_train, y_train_pred))
print("MAPE on test set : ", mean_absolute_percentage_error(y_test, y_test_pred))
print()

MSE on training set :  58550979.31889972
MSE on test set :  174328565.30609763

MAPE on training set :  0.014413385425563967
MAPE on test set :  0.03274844443963683

